# Cores & Memory: Streaming Kernels

![streaming memory](./images/ram.jpg)

## - Assignment 1 Review
## - Review: Little's Law, CPUs vs. GPUs
## - Streaming Kernels
## - Arithmetic Intensity, Machine Balance, & the Roofline Model

Each of these three concepts is critical


### Questions / confusion people have had:

#### - "Intel says this is a 6-core / 12-hardware thread package; PACE says this is a 12 core node.  Is PACE wrong?"

Try this on a compute node when you have X forwarding:

In [2]:
module load hwloc
lstopo

Machine (256GB total)
  NUMANode L#0 (P#0 128GB)
    Socket L#0 + L3 L#0 (10MB)
      L2 L#0 (256KB) + L1d L#0 (32KB) + L1i L#0 (32KB) + Core L#0 + PU L#0 (P#0)
      L2 L#1 (256KB) + L1d L#1 (32KB) + L1i L#1 (32KB) + Core L#1 + PU L#1 (P#1)
      L2 L#2 (256KB) + L1d L#2 (32KB) + L1i L#2 (32KB) + Core L#2 + PU L#2 (P#2)
      L2 L#3 (256KB) + L1d L#3 (32KB) + L1i L#3 (32KB) + Core L#3 + PU L#3 (P#3)
    HostBridge L#0
      PCIBridge
        PCI 15b3:1003
          Net L#0 "ib0"
          OpenFabrics L#1 "mlx4_0"
      PCIBridge
        PCI 10de:15f8
      PCIBridge
        PCIBridge
          PCI 1a03:2000
      PCIBridge
        PCI 8086:1521
          Net L#2 "eth0"
        PCI 8086:1521
          Net L#3 "eth1"
      PCI 8086:8d02
        Block L#4 "sda"
  NUMANode L#1 (P#1 128GB)
    Socket L#1 + L3 L#1 (10MB)
      L2 L#4 (256KB) + L1d L#4 (32KB) + L1i L#4 (32KB) + Core L#4 + PU L#4 (P#4)
      L2 L#5 (256KB) + L1d L#5 (32KB) + L1i L#5 (32KB) + Core L#5 + PU L#5 (P#5)
      L2 L

You'll see that PACE has it right because, for all of the nodes we will be using, they have installed *two sockets per node*.

### For those seeking peak GPU performance, you can also control the "grid size" (number of thread blocks)

Add, e.g., `Gs=15` to the `run_fma_prof` and `run_fma_prof_opt` targets

This will let you experiment with ILP vs. TLP.

## Review: Little's Law

$\Huge L = \lambda W$

#### - $L$: amount of $x$ in a system.
#### - $\lambda$: arrival rate, $x$ / sec.
#### - $W$: time spent in the system (sec).

- Note that it has dimensions $x$, whatever it is we're trying to count.
- For pipelines, we often think of $\lambda$ as $x$ / cycle and $W$ as length of the pipeline in cycles.  Every unit but $x$ cancels out, so we can take whichever form is more convenient.

### Example from last time: how many independent FMAs are needed for peak flop/s?

#### - $W$: depth of the pipeline
#### - $\lambda$: arrival rate = # FPUs * # FMAs in a *vectorized* instruction

[[Intel's intrisics guide](https://software.intel.com/sites/landingpage/IntrinsicsGuide/)]

![vfmadd132ps](./images/intel-intrinsics.png)

latency = $W$, CPI (cycles per instruction) = 1 / $\lambda_v$, where $\lambda_v$ = throughput for *vectorized* FMAs.  Multiply by vector width (see operation pseudocode) to get full $\lambda$.

### Question from last time:

#### - What is the latency of FMA on the GPUs?  How could we estimate it?

### Quick Review: CPUs  (Hosts)

#### - One set of instructions per thread
#### - OS schedules threads (software multithreading), can *migrate* them between cores
#### - x86-64 (AVX2) instruction set has 32 256-bit vector registers per threads
#### - Parallelism via software multithread, hardware multithreading, superscalar execution, vectorization (**SIMD**)


### Quick Review: GPUs (Devices)

(See the nice illustrations from Prof. Vuduc's [slides](http://vuduc.org/cse6230/slides/cse6230-fa14--05-cuda.pdf), starting on slide 27)

#### - A *compute kernel* is a task that the host assigns to the device in a kernel launch

```c++

solveForX<<<ThreadsPerBlock,BlocksPerGrid>>>(A,x,y);
```

- Proceeds asynchronously from the host until the host requires the results

#### - The task is broken down into a **grid** of *independent* thread blocks

- The host has no control over which thread blocks are assigned where and in what order

#### - Each thread block is assigned to a streaming multiprocessor (SM), where it stays

- A SM may be assigned multiple thread blocks

#### - The SM breaks down the thread blocks into **warps** (32 threads): a warp shares an instruction set, so all (non-divergent) instructions are vectorized
#### - The SM issues instructions from multiple warps per cycle (sometimes multiple instructions per warp per cycle) 
#### - When a warp is stalled, another is scheduled
#### - Avoid: thread divergence


### Using `nvprof` to estimate the FMA latency

`nvprof` is a performance analysis tool like `perf` and `gprof` combined for the GPU

It can be inserted before a program in the same way as `perf`.  Here's the standard invocation of our
program from the first assignment.  First info about what kind of GPU I'm using:

In [1]:
nvidia-smi

Thu Aug 29 09:44:55 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:02:00.0 Off |                    0 |
| N/A   23C    P0    25W / 250W |      0MiB / 16280MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  On   | 00000000:81:00.0 Off |                    0 |
| N/A   23C    P0    25W / 250W |      0MiB / 16280MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
        

In [2]:
cd $CSE6230_DIR/assignments/2-flops
git checkout fma_loop_dev.cu
make clean
make run_fma_prof Nh=0 Bs=1024 Gs=$((56*2)) Nd=$((1024*56*2)) T=250000

rm -f *.o *.optrpt *.so fma_prof fma_prof_opt
icc -g -Wall -std=c99 -fPIC -O3 -xHost -qopt-report=5 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_prof.o fma_prof.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -g -Wall -std=c99 -fPIC -O3 -xHost -qopt-report=5 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_omp.o fma_omp.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -g -Wall -std=c99 -fPIC -O3 -xHost -qopt-report=5 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_loop_host.o fma_loop_host.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
nvcc -ccbin=icpc -Xcompiler '-fPIC'  -dc -o fma_cuda.o fma_cuda.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -cc

Note that we are using the maximum thread block size (1024) and making two times as many thread blocks as there are SMs (56), leading to 2048 threads per SM, the maximum **occupancy**.

#### - **Occupancy**: the ratio of active warps in an SM (GPU) to the maximum number of warps per SM (GPU). 

A good measure of whether there is additional *thread-level parallelism* on the device.  Let's see if we can measure that.

(Note: for this GPU, I could get closer to 100% occupancy with a block size of 512 and twice as many blocks.  Why?)

In [3]:
nvprof --query-metrics | grep occupancy

              achieved_occupancy:  Ratio of the average active warps per active cycle to the maximum number of warps supported on a multiprocessor
              achieved_occupancy:  Ratio of the average active warps per active cycle to the maximum number of warps supported on a multiprocessor


Let's try it out:

In [5]:
make run_fma_prof Nh=0 Bs=512 Gs=$((56*4)) Nd=$((1024*56*2)) T=250000 PERF="nvprof --metrics achieved_occupancy"

OMP_PROC_BIND=spread OMP_NUM_THREADS=1 nvprof --metrics achieved_occupancy ./fma_prof 0 114688 512 224 250000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 114688, T = 250000, block size = 512
==22377== NVPROF is profiling process 22377, command: ./fma_prof 0 114688 512 224 250000 0.5 3.0
[./fma_prof]: 5.339694e-02 elapsed seconds
[./fma_prof]: 114688000000 flops executed
[./fma_prof]: 2.147838e+12 flop/s
==22377== Profiling application: ./fma_prof 0 114688 512 224 250000 0.5 3.0
==22377== Profiling result:
==22377== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "Tesla P100-PCIE-16GB (0)"
    Kernel: fma_loop_dev(int, int, float*, float, float)
          1                        achieved_occupancy                        Achieved Occupancy    0.976454    0.976454    0.976454
    Kernel: __nv_static_56__43_tmpxft_000052c9_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf
          1  

Okay, but let's look at the measure that we're actually interested, the percentag of peak flop/s

In [15]:
nvprof --query-metrics | grep flop | grep peak

              flop_hp_efficiency:  Ratio of achieved to peak half-precision floating-point operations
              flop_sp_efficiency:  Ratio of achieved to peak single-precision floating-point operations
              flop_dp_efficiency:  Ratio of achieved to peak double-precision floating-point operations
              flop_hp_efficiency:  Ratio of achieved to peak half-precision floating-point operations
              flop_sp_efficiency:  Ratio of achieved to peak single-precision floating-point operations
              flop_dp_efficiency:  Ratio of achieved to peak double-precision floating-point operations


In [6]:
make run_fma_prof Nh=0 Bs=1024 Gs=$((56*2)) Nd=$((1024*56*2)) T=250000 PERF="nvprof --metrics flop_sp_efficiency"

OMP_PROC_BIND=spread OMP_NUM_THREADS=1 nvprof --metrics flop_sp_efficiency ./fma_prof 0 114688 1024 112 250000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 114688, T = 250000, block size = 1024
==23105== NVPROF is profiling process 23105, command: ./fma_prof 0 114688 1024 112 250000 0.5 3.0
==23105== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==23105== Replaying kernel "__nv_static_56__43_tmpxft_000052c9_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf" (done)           
==23105== Some kernel(s) will be replayed on device 1 in order to collect all events/metrics.
==23105== Replaying kernel "__nv_static_56__43_tmpxft_000052c9_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf" (done)           
==23105== Replaying kernel "fma_loop_dev(int, int, float*, float, float)" (done)           
==23105== Replaying kernel "fma_loop_dev(int, int, float*, float, float)" (done)           
[./fma_prof]: 3.038997e+00 elapsed seconds
[./fma_prof]: 11468800

So while we have good occupancy, we are acheiving only ~50% of peak flop/s.

First, let's unroll things just a little bit.

In [7]:
sed -i -e '/for (int j/i #pragma unroll 64' fma_loop_dev.cu
git diff fma_loop_dev.cu

diff --git a/assignments/2-flops/fma_loop_dev.cu b/assignments/2-flops/fma_loop_dev.cu
index c603504..d556288 100644
--- a/assignments/2-flops/fma_loop_dev.cu
+++ b/assignments/2-flops/fma_loop_dev.cu
@@ -9,6 +9,7 @@ fma_loop_dev (int N, int T, float *a, float b, float c)
   int num_threads = gridDim.x * blockDim.x;
 
   for (int i = my_thread; i < N; i+= num_threads) {
+#pragma unroll 64
     for (int j = 0; j < T; j++) {
       a[i] = a[i] * b + c;
     }


In [8]:
make run_fma_prof Nh=0 Bs=1024 Gs=$((56*2)) Nd=$((1024*56*2)) T=250000 PERF="nvprof --metrics flop_sp_efficiency"

nvcc -ccbin=icpc -Xcompiler '-fPIC'  -dc -o fma_loop_dev.o fma_loop_dev.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -dlink  fma_cuda.o fma_loop_dev.o -o fma_cuda_link.o
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
icpc -qopenmp -shared -Wl,-soname,libfma_cuda.so -o libfma_cuda.so fma_cuda_link.o fma_cuda.o fma_loop_dev.o -L/usr/local/pacerepov1/cuda/8.0.44/lib64 -Wl,-rpath,/usr/local/pacerepov1/cuda/8.0.44/lib64 -lcudart
icpc -qopenmp -o fma_prof fma_prof.o fma_omp.o fma_loop_host.o libfma_cuda.so -Wl,-rpath,.
OMP_PROC_BIND=spread OMP_NUM_THREADS=1 nvprof --metrics flop_sp_efficiency ./fma_prof 0 114688 1024 112 250000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 114688, T = 250000, block size = 

Okay, now we're seeing about ~90% efficiency (thought it took a lot of unrolling!)

Now, according to the chart from last time:

![NVIDIA comparison](../processors/images/nvidia-table.png)

There are 64 single precision FPUs per SM on this Pascal GPU.  If I keep the same number of thread blocks, but set each block size to $64 / 2= 32$ (1 warp!), then there will be one thread per FPU.

Because each thread's computation has a loop dependency (iteration $i + 1$ cannot start until iteration $i$ has completed), we should only be issuing one FMA at the rate that an operation can traverse the pipeline.  Let's see:

In [9]:
make run_fma_prof Nh=0 Bs=32 Gs=$((56*2)) Nd=$((32*56*2)) T=8000000 PERF="nvprof --metrics flop_sp_efficiency --metrics achieved_occupancy"

OMP_PROC_BIND=spread OMP_NUM_THREADS=1 nvprof --metrics flop_sp_efficiency --metrics achieved_occupancy ./fma_prof 0 3584 32 112 8000000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 3584, T = 8000000, block size = 32
==24563== NVPROF is profiling process 24563, command: ./fma_prof 0 3584 32 112 8000000 0.5 3.0
==24563== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==24563== Replaying kernel "__nv_static_56__43_tmpxft_000052c9_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf" (done)           
==24563== Some kernel(s) will be replayed on device 1 in order to collect all events/metrics.
==24563== Replaying kernel "__nv_static_56__43_tmpxft_000052c9_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf" (done)           
==24563== Replaying kernel "fma_loop_dev(int, int, float*, float, float)" (done)           
==24563== Replaying kernel "fma_loop_dev(int, int, float*, float, float)" (done)           
[./fma_prof]: 1.874314e+01 elapsed seconds
[.

Note that our achieved occupancy is ~1/32 = 32 / 1024, which makes sense.  Our efficiency is ~16%, so our estimated pipeline depth is **$1 / 0.16 \approx 6$**.

You could repeat this for the Kepler GPUs if you wanted, but the number would be quite different.
As the chart above shows, the design of NVIDIA GPUs is not stable: Kepler has a few large SMs; Pascal has a lot of smaller SMs; Maxwell (not available on pace-ice) is in the middle, but almost completely abandons double precision arithmetic, which makes me sad :(.

We're about to expand the types of programs we look at from those whose performance depends only on compute power to those whose performance also depends on the computers ability to move data to and from the processors.  We start with the simplest type of memory movement possible.

## Streaming Kernels

Sometimes our problems take the form of pure and simple *data parallelism*:

- We have some kernel operation $f$ that has some mix of $\color{blue}{inputs}$ and $\color{red}{outputs}$ (some times a piece of data can be both).  Consider the example of
  computing a particle's kinetic energy from it's momentum:
  
  $$\Large \color{red}{e} \leftarrow f(\color{blue}{m, u, v, w}) = \frac{1}{2} m (u^2 + v^2 + w^2).$$
  
- If we want to do this more multiple particles, they can be computed independently:

  $$\large \color{red}{e_i} \leftarrow f(\color{blue}{m_i, u_i, v_i, w_i}) = \frac{1}{2} m_i (u_i^2 + v_i^2 + w_i^2).$$

Assuming we were processing each one from *registers*, how long would it take to compute the energy for $N$ particles?  Assuming $N$ is large, we would like that as a rate: **particle energies computed per second**.

This is what last lecture was about:

- **Simple upper bound (count the operations):** Divide the flop/s of whatever compute resource we have by the number of flops in the kernel.  Note the two terms in this ratio:

  - flop/s in the *machine*
  - flops in the *kernel* (algorithm)

- **Pipeline based estimate (complicated):** Find the critical path; compute its length in pipeline cycles.  This gives the latency for computing one particle.  This should be a *lower bound*:
  - If the whole kernel can be *vectorized*, then we can multiply by the vector width.
  - If there are bubbles in the computation of one kernel that can be filled by the computation of another (ILP),
    leading to a throughput that is higher than the inverse of latency
    
`TODO: whiteboard`
    
- **Measure (seems easy...):** Set up a bunch of particles and see how long it takes.

### Note that I'm asking you to phrase your rate in machine independent units

When people want to compare algorithms and machines for streaming kernels, the comparison that matters is the rate at which real things are accomplished.

Let's say my kernel operation looks like lots of small sparse matrix-vector multiplications:

- Algorithm A on machine 1 achieves 2 Tflop/s, but the way it computes its kernel is dense linear, and most of the operations that it performs are $c \leftarrow c + 0*0$.

- Algorithm B on machine 2 achives 2 Gflops/s, uses a sparse matrix format which requires more complicated instructions: it achieves a small fraction of Gflop/s, but each FMA is meaningful work $c \leftarrow c + a *  b$.

Which is better?  I haven't given you enough information to tell, because I've only given you machine dependent numbers!

### Rules of thumb about measurement:

#### - Usually we want to test something that is important, which means we do it over and over again

Unless you know otherwise, assume that the thing you are trying to measure over and over again
has *side effects* on the system that it is running on: that the system is adapting to your repetition.

(Example we've already discussed: branch prediction)

Because of that, it's usually good practice to **discard the first few timings**

#### - Make your measurements reproducible

Not only keep track of which version of the code you are measuring, but

- how it was configured/compiled
- what the environment variables were that affect it

#### - Be aware of the precision of your timer relative to what you are trying to time

If your timer calls a function (overhead of callstack operations, movement to and from memory, etc.) and you're measuring something that only lasts a few cycles, you will be measuring noise

In [13]:
make run_fma_prof Nh=10000000000 Nd=0 T=1

icc -g -Wall -std=c99 -fPIC -O3 -xHost -qopt-report=5 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_loop_host.o fma_loop_host.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icpc -qopenmp -o fma_prof fma_prof.o fma_omp.o fma_loop_host.o libfma_cuda.so -Wl,-rpath,.
OMP_PROC_BIND=spread OMP_NUM_THREADS=1  ./fma_prof 10000000000 0 -1 -1 1 0.5 3.0
[./fma_prof] Nh = 1410065408, Nd = 0, T = 1, default block size
[./fma_prof]: 5.892210e-01 elapsed seconds
[./fma_prof]: 2820130816 flops executed
[./fma_prof]: 4.786202e+09 flop/s


In [14]:
objdump -Sd fma_loop_host.o | pygmentize -l c-objdump

fma_loop_host.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <fma_loop_host>:
 * Input-Outputs:
 * a : the array
 */
void
fma_loop_host (int N, int T, float *a, float b, float c)
{
   0:	48 89 d1             	mov    %rdx,%rcx
  for (int i = 0; i < N; i++) {
   3:	85 ff                	test   %edi,%edi
   5:	0f 8e 05 01 00 00    	jle    110 <fma_loop_host+0x110>
   b:	83 ff 10             	cmp    $0x10,%edi
   e:	0f 8c 04 01 00 00    	jl     118 <fma_loop_host+0x118>
  14:	48 89 c8             	mov    %rcx,%rax
  17:	48 83 e0 1f          	and    $0x1f,%rax
  1b:	89 c0                	mov    %eax,%eax
  1d:	85 c0                	test   %eax,%eax
  1f:	74 10                	je     31 <fma_loop_host+0x31>
  21:	a8 03                	test   $0x3,%al
  23:	0f 85 ef 00 00 00    	jne    118 <fma_loop_host+0x118>
  29:	f7 d8                	neg    %eax
  2b:	83 c0 20             	add    $0x20,%eax
  2e:	c1 e8 02             	shr    $0x2,%eax
  31:	8d 50 10    

{
  for (int i = 0; i < N; i++) {
  b4:	89 f8                	mov    %edi,%eax
  b6:	83 e0 fc             	and    $0xfffffffffffffffc,%eax
  b9:	45 33 c0             	xor    %r8d,%r8d
  bc:	48 63 c0             	movslq %eax,%rax
 *
 * Input-Outputs:
 * a : the array
 */
void
fma_loop_host (int N, int T, float *a, float b, float c)
  bf:	c4 e2 79 18          	(bad)  
  c3:	d9 c4                	fld    %st(4)
  c5:	e2 79                	loop   140 <fma_loop_host+0x140>
  c7:	18 d0                	sbb    %dl,%al
{
  for (int i = 0; i < N; i++) {
    a[i] = a[i] * b + c;
  c9:	48 8d 34 91          	lea    (%rcx,%rdx,4),%rsi
 * a : the array
 */
void
fma_loop_host (int N, int T, float *a, float b, float c)
{
  for (int i = 0; i < N; i++) {
  cd:	49 83 c0 04          	add    $0x4,%r8
    a[i] = a[i] * b + c;
  d1:	c5 f8 10 26          	vmovups (%rsi),%xmm4
  d5:	c4 e2 69 a8 e3       	vfmadd213ps %xmm3,%xmm2,%xmm4
  da:	c5 f8 11 26          	vmovups %xmm4,(%rsi)
 * a : the array
 */
void
fma_

### What did we see (hopefully):

- `fma_loop_host`, with $T$ large out performs `fma_loop_host`, with $T=1$, even when we hard code and achieve the same vectorization, because with $T=1$ there is one load and store
one **load** and one **store** for each FMA.


## Load, Store, and Main Memory


#### How long does a load take?  How long does a store take?

This depends on whether the data is in *cache*.

Today we are considering the rate at which we can apply a streaming kernel to a lot of data: so much that, even if we were using the data before, only a negligible percentage would already be in cache.

**For the asymptotic rate of a streaming kernel, we care only about data that was originally in main memory, not in cache**

Therefore, the numbers we need to know are:

- The *latency* of moving data from memory to compute (secs)
- The *streaming bandwidth* of moving data from memory to compute (bytes / sec)
- The *bytes loaded* and *bytes stored* per kernel

## A look at the hardware side

[Hager and Wellein, Slides 68-82](https://moodle.rrze.uni-erlangen.de/pluginfile.php/12220/mod_resource/content/10/01_Arch.pdf)

What did we learn?

- Even if we only request a small amount of data in a load or write a small amount in a store, data moves to and from memory in increment of a *cache line*.  Achieving peak throughputs is only possible if you are accessing memory in sequence.

  - One of our first examples in this class of how hardware rewards **locality**: using pieces of data that are close in space (location in memory) and time (one right after the other).
  
  - The same principles apply to GPUs, except that we have to think of the load of each thread as a vectorized load
    of multiple pieces of data.  The loads from threads in a warp issue at the same time if they address 
    *consecutive locations* in memory.  (Called "coalesced memory access").  Uncoalesced memory access can get 
    serialized as multiple load instructions.
    
- The same principles as our previous lecture on processors alone apply: we don't see the latency of the memory transfer for large data sets because *hardware prefetching* is the pipeline parallelism of memory access.

  - **One key difference:** If we want to model this as a pipeline, it is a heterogeneous pipeline.
    - We cannot diagram them as boxes taking equal amount of time as before.
    - The hardware units of throughput on the different components of the pipeline are different: one is
      byte/s, the other is flop/s.  **How do we determine what the throughput rate of the full pipeline is?**
    

## A Benchmark for the hardware side: the STREAM benchmark

[John D. McCalpin's STREAM slides](http://sites.utexas.edu/jdm4372/2016/11/22/sc16-invited-talk-memory-bandwidth-and-system-balance-in-hpc-systems/)

`TODO if time: live demonstration`

## Putting it all together

The throughput rate is going to depend on a ratio of ratios:

- **Arithmetic Intensity:** (flops / kernel) / (bytes loaded and stored / kernel).  Units are flop / byte.

- **Machine Balance:** (peak flop/s) / (peak byte/s).  Units are flop / byte.

- These combine for the **roofline model**

[Prof. Vuduc's GPU tuning slides, 12-26](http://vuduc.org/cse6230/slides/cse6230-fa14--07-gpu-tuning-1.pdf)